# BOSS Textract NER

Você está preste a utilizar o [BOSS Textract NER](https://github.com/RyanPrado/BOSS-Textract-NER), lembre-se que para utiliza-lo, um modelo treino é necessário, caso não tenha treinado um modelo ainda, de uma olhada [aqui](https://github.com/RyanPrado/BOSS-Textract-NER/blob/main/README.md), abaixo você verá uma inicialização de uso em massa, sinta-se a vontade para modificar como desejar!

## Adicionando a aplicação para ser lida pelo Kernel

In [ ]:
import os
import sys
os.chdir(os.path.expanduser("../"))

## Importando bibliotecas necessárias para utilização deste notebook

In [ ]:
from IPython.core.display import display, HTML
from IPython.display import IFrame
import time
from pathlib import Path
from boss_textract.utils import SEPARATORS
from boss_textract.utils.logger import logger
import pandas as pd
import re
TODAY_TIME=time.strftime('%d-%m-%Y_%Hh%M')

## Declarando variáveis de utilização
* **input_path** `<dir|file|glob>` : Caminho para acessar o(s) arquivo(s) para ser(em) processado(s);
* **output_path** `<dir|file>` : Caminho de saída do(s) arquivo(s) processado(s);
* **model_path** `<dir>` : Diretório do modelo a ser utilização para predição;
* **source_column** `<str>` : Nome da coluna de origem dentro do(s) arquivo(s) a ser(em) processado(s);
* **output_column** `<str>` : Nome da coluna de saída a ser criada dentro do(s) arquivo(s) a ser(em) processado(s);
* **separator** `<str>` : Separator a ser utilizado pelo(s) arquivo(s) [TAB;COMMA;SEMICOLON;PIPE];
* **dictionary_file** `<file>` : Arquivo de dicionário a ser utilizado no pós-processamento;

### dictionary_file

Este arquivo deve ser um `.csv` separado por `;`, deve conter duas colunas `FROM` e `TO`:
* `FROM` - Pode ser um REGEX para realizar a verificação nos textos tanto de origem quanto de saída, caso o texto de saída seja vazio, ele procurará no texto de origem, também pode receber um `tag`;
* `TO` - Texto de saída, caso o padrão de `FROM` for encontrado este texto sumirá a posição do texto de saída;

#### TAGs
`<INVOICE value='{INVOICE NUMBER}>'` - Este é um exemplo de tag que `FROM` pode receber, substitua `{INVOICE NUMBER}` pelo valor ou por um regex, sua esquema de leitura poderá ser feito mais abaixo na função [`def process_dictionary`](App.ipynb#declarando-função-de-pós-processamento)

In [ ]:
input_path="<WRITE HERE>"       #   Ex. /home/usr/example_dir/*.csv
output_path="<WRITE HERE>"      #   Ex. /home/usr/example_dir_output
model_path="<WRITE HERE>"       #   Ex. /home/usr/models/best-model
source_column="<WRITE HERE>"    #   Ex. SOURCE
output_column="<WRITE HERE>"    #   Ex. SOURCE_RESPONSE
separator="<WRITE HERE>"        #   Ex. TAB
dictionary_file="<WRITE HERE>"  #   Ex. /home/user/dictionary_dir/my_dict.csv

## Convertendo variáveis & lendo arquivos a serem processados

In [ ]:
output_path = Path(output_path)
if output_path.name != f"NER_{TODAY_TIME}":
    output_path = output_path / f"NER_{TODAY_TIME}"
if not output_path.exists():
    output_path.mkdir(parents=True, exist_ok=True)
    
model_path = Path(model_path)

files_paths = []
if os.name == "posix" and input_path[0] == "/":
    files_paths = list(Path('/').glob(input_path[1:]))
elif os.name == "nt" and re.search(r"[aA-zZ]\:(\\|//)",input_path):
    input_path = re.sub(r"\\","/",input_path)
    files_paths = list(Path(f'{input_path[0]}:/').glob(input_path[3:]))
else:
    input_path = list(Path('.').glob(input_path))

logger.info(f"File(s) processing: {len(files_paths)}")

## Realizando predição dos arquivos através do modelo

In [ ]:
for file in files_paths:
    try:
        file_path = file.absolute()
        logger.info(f"Starting processing: {file_path}")
        !uv run boss_textract predict --data="$file_path" --model="$model_path" --output="$output_path" --gpu_id=0 --src_col="$source_column" --out_col="$output_column" --sep="$separator" --no-log
    except Exception as e:
        exception = sys.exc_info()
        logger.opt(exception=exception).error(e)

## Declarando função de pós-processamento

A função abaixo realiza o pós-processamento baseado no dicionario informado anteriormente, seu objetivo é limpar pois sujeiras no resultados e também realizar um formatação para alguns itens que não podem ser bem extraídos pelo modelo.
`df_regex` - é referenciado pela variável `dictionary_file` porém aqui o arquivo foi lido como um `pd.DataFrame`.

In [ ]:
def process_dictionary(row, df_regex):
    for _, regex_row in df_regex.iterrows():
        from_text = regex_row['FROM']
        to_text = regex_row['TO']
        source_text = row[source_column]
        output_text = row[output_column]
        is_invoice = False

        if re.search(r"\<INVOICE\svalue=\'.*\'\>",from_text):
            start_idx = from_text.find("value='") + 7
            end_idx = from_text.find("'",start_idx)
            invoice = from_text[start_idx:end_idx]
            from_text = r"(MEMO)?(?(1)\s{invoice}|INV(?:OICE)?(?:\.|\s\-)?\s?(?:{invoice}))".format(invoice=invoice)
            is_invoice = True

        if isinstance(to_text, str):
            to_text = to_text.upper()
        if isinstance(source_text, str):
            source_text = source_text.upper()
        if isinstance(output_text, str):
            output_text = output_text.upper()

        if (is_invoice and re.search(from_text, source_text, re.IGNORECASE)) or (not is_invoice and pd.isna(output_text) and re.search(from_text, source_text, re.IGNORECASE)) or (not is_invoice and not pd.isna(output_text) and re.search(from_text, output_text, re.IGNORECASE)):
            row[output_column] = to_text
            break
    return row

In [ ]:
df_regex = pd.read_csv(Path(dictionary_file).absolute(), sep=";", encoding="utf-8",index_col=False)

## Pós Processamento dos arquivos

Realiza o pós processamento em todos os arquivos presentes no diretório de saída

In [ ]:
for file in output_path.iterdir():
    logger.info(f"Pós Processando: {file.absolute()}")
    try:
        df = pd.read_csv(file.absolute(), sep=SEPARATORS.get(separator),encoding="utf-8",index_col=False)
        df = df.apply(process_dictionary, axis=1, args=(df_regex,))
        logger.success(f"Pós Processado: {file.absolute()}")
        df.to_csv(file.absolute(),sep=SEPARATORS.get(separator),encoding="utf-8",index=False)
        del df
    except Exception as e:
        exception = sys.exc_info()
        logger.opt(exception=exception).error(e)